In [81]:
#!conda install pyspark
#!conda install findspark
#!conda install kagglehub
#!conda install openjdk=8


In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .master("local[*]") \
    .getOrCreate()

# Test with a simple DataFrame
data = [("John", 30), ("Alice", 25), ("Bob", 35)]
df = spark.createDataFrame(data, ["Name", "Age"])
df.show()

24/12/02 14:35:00 WARN Utils: Your hostname, Ruikes-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.69 instead (on interface en0)
24/12/02 14:35:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/02 14:35:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+---+
| Name|Age|
+-----+---+
| John| 30|
|Alice| 25|
|  Bob| 35|
+-----+---+



In [2]:
import kagglehub
path = "/Users/rzhang/.cache/kagglehub/datasets/microize/newyork-yellow-taxi-trip-data-2020-2019/versions/25"
# uncomment the following line to download the file package, size = 10GB
#path = kagglehub.dataset_download(
#    "microize/newyork-yellow-taxi-trip-data-2020-2019"
#)
print("Path to dataset files:", path)


Path to dataset files: /Users/rzhang/.cache/kagglehub/datasets/microize/newyork-yellow-taxi-trip-data-2020-2019/versions/25


/opt/anaconda3/envs/spark_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#df = spark.read.csv(path + "/yellow_tripdata_2019-06.csv", header=True, inferSchema=True)
df = spark.read.csv("yellow_tripdata_2019-06.csv", header=True, inferSchema=True) \
    .sample(withReplacement=False, fraction=0.01, seed=42)
df.head(5)

[Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2019, 6, 1, 0, 2, 40), tpep_dropoff_datetime=datetime.datetime(2019, 6, 1, 0, 18, 37), passenger_count=1, trip_distance=2.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=68, payment_type=1, fare_amount=11.5, extra=3.0, mta_tax=0.5, tip_amount=2.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=17.3, congestion_surcharge=2.5),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2019, 6, 1, 0, 22, 43), tpep_dropoff_datetime=datetime.datetime(2019, 6, 1, 0, 27, 27), passenger_count=1, trip_distance=1.53, RatecodeID=1, store_and_fwd_flag='N', PULocationID=48, DOLocationID=239, payment_type=1, fare_amount=6.5, extra=0.5, mta_tax=0.5, tip_amount=2.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=12.3, congestion_surcharge=2.5),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2019, 6, 1, 0, 21), tpep_dropoff_datetime=datetime.datetime(2019, 6, 1, 0, 30, 49), passenger_count=1, tr

In [4]:
#print all columns
df.columns



['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

 Column Name:
 
 1. 'VendorID'  TPEP Provider ID 1 or 2, not used in this project
 2. 'tpep_pickup_datetime'  Pickup date and time
 3. 'tpep_dropoff_datetime'  Dropoff date and time
 4. 'passenger_count'  Number of passengers
 5. 'trip_distance'  Trip distance in miles
 6. 'RatecodeID'  Rate code ID: 1=Standard rate, 2=JFK, 3=Newark, 4=Nassau or Westchester, 5=Negotiated fare, 6=Group ride
 7. 'store_and_fwd_flag'  Store and forward flag, not used in this project
 8. 'PULocationID'  Pickup location ID
 9. 'DOLocationID'  Dropoff location ID
 10. 'payment_type'  Payment type: 1=Credit card, 2=Cash, 3=No charge, 4=Dispute, 5=Unknown, 6=Voided trip
 11. 'fare_amount'  Fare amount
 12. 'extra'  Extra charges
 13. 'mta_tax'  MTA tax
 14. 'tip_amount'  Tip amount
 15. 'tolls_amount'  Tolls amount
 16. 'improvement_surcharge'  Improvement surcharge
 17. 'total_amount'  Total amount
 18. 'congestion_surcharge'  Congestion surcharge

In [5]:
# Print the schema (column names and types)
df.printSchema()

# Get number of rows
print("Number of rows:", df.count())

# Show basic statistics for numeric columns
df.describe().show()

# Show the first few rows
df.show(5)

# Get column names
print("Columns:", df.columns)

# Get basic data types of columns
for col in df.columns:
    print(f"{col}: {df.schema[col].dataType}")


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



24/12/02 14:35:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Number of rows: 69859


+-------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+------------------+---------------------+------------------+--------------------+
|summary|          VendorID|   passenger_count|     trip_distance|        RatecodeID|store_and_fwd_flag|      PULocationID|     DOLocationID|       payment_type|       fare_amount|             extra|            mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+------------------+---------------------+------------------+--------------------+
|  count|             69859|            

In [6]:
# Drop specified columns
df_cleaned = df.drop(
    'VendorID',
    'store_and_fwd_flag',
    'fare_amount',
    'extra',
    'mta_tax',
    'tolls_amount',
    'improvement_surcharge',
    'congestion_surcharge',
    'payment_type',
    'tip_amount'
)

# Verify the remaining columns
print("Remaining columns:", df_cleaned.columns)

# Show a few rows of the cleaned dataset
df_cleaned.show(5)

Remaining columns: ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID', 'total_amount']
+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|total_amount|
+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+
| 2019-06-01 00:02:40|  2019-06-01 00:18:37|              1|          2.0|         1|         162|          68|        17.3|
| 2019-06-01 00:22:43|  2019-06-01 00:27:27|              1|         1.53|         1|          48|         239|        12.3|
| 2019-06-01 00:21:00|  2019-06-01 00:30:49|              1|          1.5|         1|         148|          87|        13.8|
| 2019-06-01 00:39:45|  2019-06-01 00:48:40|              1|         1.28|         

In [7]:
# for better machine learning result, replace pullocationID and dolocationID with location name

df_zone_name = spark.read.csv('taxi+_zone_lookup.csv', header=True, inferSchema=True)
df_zone_name = df_zone_name.drop('Borough', 'service_zone')
df_zone_name.show(5)


+----------+--------------------+
|LocationID|                Zone|
+----------+--------------------+
|         1|      Newark Airport|
|         2|         Jamaica Bay|
|         3|Allerton/Pelham G...|
|         4|       Alphabet City|
|         5|       Arden Heights|
+----------+--------------------+
only showing top 5 rows



In [8]:


# 1. First approach: Using RDD operations (closer to traditional MapReduce)
# Convert location mapping to dictionary
location_dict = dict(df_zone_name.select("LocationID", "Zone").rdd.collect())

# Create broadcast variable for the mapping
location_broadcast = spark.sparkContext.broadcast(location_dict)

# Define map functions
def map_locations(row):
    locations = location_broadcast.value
    # Map both pickup and dropoff locations
    pu_location = locations.get(row.PULocationID, "Unknown")
    do_location = locations.get(row.DOLocationID, "Unknown")
    
    return (
        row.tpep_pickup_datetime,
        row.tpep_dropoff_datetime,
        row.passenger_count,
        row.trip_distance,
        row.RatecodeID,
        pu_location,  # Replaced PULocationID
        do_location,  # Replaced DOLocationID
        row.total_amount
    )

# Apply the transformation

# 1. Convert DataFrame to RDD
rdd = df_cleaned.rdd

# 2. Apply the mapping function
mapped_rdd = rdd.map(map_locations)

# 3. Define the schema (column names for the new DataFrame)
new_column_names = [
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'passenger_count',
    'trip_distance',
    'RatecodeID',
    'pickup_location',
    'dropoff_location',
    'total_amount'
]

# 4. Convert back to DataFrame with the new schema
df_mapped = mapped_rdd.toDF(new_column_names)

# Show results
df_mapped.show(5)

+--------------------+---------------------+---------------+-------------+----------+--------------------+--------------------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|     pickup_location|    dropoff_location|total_amount|
+--------------------+---------------------+---------------+-------------+----------+--------------------+--------------------+------------+
| 2019-06-01 00:02:40|  2019-06-01 00:18:37|              1|          2.0|         1|        Midtown East|        East Chelsea|        17.3|
| 2019-06-01 00:22:43|  2019-06-01 00:27:27|              1|         1.53|         1|        Clinton East|Upper West Side S...|        12.3|
| 2019-06-01 00:21:00|  2019-06-01 00:30:49|              1|          1.5|         1|     Lower East Side|Financial Distric...|        13.8|
| 2019-06-01 00:39:45|  2019-06-01 00:48:40|              1|         1.28|         1|Greenwich Village...|           Hudson Sq|       13.56|
| 2019-06-01 

In [9]:
from pyspark.sql.functions import hour, to_date

# Create new DataFrame with extracted date and hour
df_time_processed = df_mapped \
    .withColumn('pickup_date', to_date('tpep_pickup_datetime')) \
    .withColumn('pickup_hour', hour('tpep_pickup_datetime')) \
    .withColumn('dropoff_date', to_date('tpep_dropoff_datetime')) \
    .withColumn('dropoff_hour', hour('tpep_dropoff_datetime')) \
    .drop('tpep_pickup_datetime', 'tpep_dropoff_datetime')

# Reorder columns for better readability
columns_order = [
    'pickup_date',
    'pickup_hour',
    'dropoff_date',
    'dropoff_hour',
    'passenger_count',
    'trip_distance',
    'RatecodeID',
    'pickup_location',
    'dropoff_location',
    'total_amount'
]

df_time_processed = df_time_processed.select(columns_order)

# Show the result
df_time_processed.show(5)

+-----------+-----------+------------+------------+---------------+-------------+----------+--------------------+--------------------+------------+
|pickup_date|pickup_hour|dropoff_date|dropoff_hour|passenger_count|trip_distance|RatecodeID|     pickup_location|    dropoff_location|total_amount|
+-----------+-----------+------------+------------+---------------+-------------+----------+--------------------+--------------------+------------+
| 2019-06-01|          0|  2019-06-01|           0|              1|          2.0|         1|        Midtown East|        East Chelsea|        17.3|
| 2019-06-01|          0|  2019-06-01|           0|              1|         1.53|         1|        Clinton East|Upper West Side S...|        12.3|
| 2019-06-01|          0|  2019-06-01|           0|              1|          1.5|         1|     Lower East Side|Financial Distric...|        13.8|
| 2019-06-01|          0|  2019-06-01|           0|              1|         1.28|         1|Greenwich Village...

In [10]:
from pyspark.sql.functions import date_format, dayofweek

# Add day of week (1 = Sunday, 7 = Saturday)
df_with_dow = df_time_processed \
    .withColumn('pickup_day_of_week', date_format('pickup_date', 'EEEE')) \
    .withColumn('dropoff_day_of_week', date_format('dropoff_date', 'EEEE'))



# Reorder columns
columns_order = [
    'pickup_date',
    'pickup_day_of_week',
    'pickup_hour',
    'dropoff_date',
    'dropoff_day_of_week',
    'dropoff_hour',
    'passenger_count',
    'trip_distance',
    'RatecodeID',
    'pickup_location',
    'dropoff_location',
    'total_amount'
]

df_with_dow = df_with_dow.select(columns_order)

# Show the result
from pyspark.storagelevel import StorageLevel
#df_with_dow.persist(StorageLevel.MEMORY_ONLY)
df_with_dow.cache()
df_with_dow.show(5)

+-----------+------------------+-----------+------------+-------------------+------------+---------------+-------------+----------+--------------------+--------------------+------------+
|pickup_date|pickup_day_of_week|pickup_hour|dropoff_date|dropoff_day_of_week|dropoff_hour|passenger_count|trip_distance|RatecodeID|     pickup_location|    dropoff_location|total_amount|
+-----------+------------------+-----------+------------+-------------------+------------+---------------+-------------+----------+--------------------+--------------------+------------+
| 2019-06-01|          Saturday|          0|  2019-06-01|           Saturday|           0|              1|          2.0|         1|        Midtown East|        East Chelsea|        17.3|
| 2019-06-01|          Saturday|          0|  2019-06-01|           Saturday|           0|              1|         1.53|         1|        Clinton East|Upper West Side S...|        12.3|
| 2019-06-01|          Saturday|          0|  2019-06-01|        

In [11]:
#count unique dropoff_hour
df_with_dow.select('dropoff_hour').distinct().count()

#export to csv
df_with_dow.write.csv('df_with_dow.csv', header=True, mode='overwrite')


In [12]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import struct, col

# 1. Convert DataFrame to RDD and map to (location_pair, 1) format
location_pairs_rdd = df_with_dow.rdd.map(
    lambda row: (
        (row.pickup_location, row.dropoff_location), 
        1
    )
)

# 2. Reduce by key (location pair) to sum frequencies
location_pairs_count = location_pairs_rdd.reduceByKey(lambda x, y: x + y)

# 3. Sort by frequency in descending order
# FIXED: Changed sortByKey to sortBy and added proper sorting function
sorted_pairs = location_pairs_count.sortBy(lambda x: x[1], ascending=False)

# 4. Convert back to DataFrame for better display
result_df = sorted_pairs.toDF(['location_pair', 'frequency'])
#result_df = location_pairs_count.toDF(['location_pair', 'frequency'])

# 5. Show top results
#print("Most common pickup-dropoff location pairs:")
result_df.show(10, truncate=False)

# Optional: Get total number of unique pairs
print("\nTotal unique location pairs:", result_df.count())

+----------------------------------------------+---------+
|location_pair                                 |frequency|
+----------------------------------------------+---------+
|{NV, NV}                                      |476      |
|{Upper East Side South, Upper East Side North}|430      |
|{Upper East Side South, Upper East Side South}|354      |
|{Upper East Side North, Upper East Side South}|333      |
|{Upper East Side North, Upper East Side North}|332      |
|{Upper East Side South, Midtown East}         |197      |
|{Upper East Side South, Midtown Center}       |189      |
|{Lincoln Square East, Upper West Side South}  |185      |
|{Upper West Side South, Upper West Side North}|183      |
|{Upper West Side North, Upper West Side South}|181      |
+----------------------------------------------+---------+
only showing top 10 rows


Total unique location pairs: 6544


In [13]:
from pyspark.sql import SparkSession
SparkSession.builder.getOrCreate().stop()